## Make MOSES easier to use for the VSCode Deniers

In [1]:
# run this cell so changes outside here are automatically detected
%load_ext autoreload
%autoreload 2

In [18]:
import sys
sys.path.append("../")

from algorithm.lattice.Lattice import Lattice
from algorithm.lattice.Voxel import Voxel
from algorithm.Moses import Moses

In [24]:
import pandas as pd

class LatticeLoader:
    def __init__(self):
        pass

    @staticmethod
    def create_excel(nx:int, ny:int, nz:int, is_unit_cell: bool, filename="lattice.xlsx"):
        # --- 1. build config sheet ---
        config_df = pd.DataFrame({
            "param": ["nx", "ny", "nz", "is_unit_cell"],
            "value": [nx, ny, nz, bool(is_unit_cell)]
        })

        # --- 2. build lattice sheet ---
        rows = []

        # HEADER ROW: section titles
        header = ["Lattice Coordinates"] + [""] * (nx - 1)
        header += ["", "Cargo Material"] + [""] * (nx - 1)
        header += ["", "Cargo Coordinates"] + [""] * (nx - 1)
        rows.append(header)

        # for each layer (z) from top to bottom
        for z in reversed(range(nz)):
            # layer label row
            layer_row = []
            # lattice coordinates block
            layer_row.extend([f"Layer {z}"] + [""] * (nx - 1))
            layer_row.append("")  
            # cargo material block
            layer_row.extend([f"Layer {z}"] + [""] * (nx - 1))
            layer_row.append("")  
            # cargo coordinates block
            layer_row.extend([f"Layer {z}"] + [""] * (nx - 1))
            rows.append(layer_row)

            # for each y row in this layer (top -> bottom)
            for y in reversed(range(ny)):
                row = []
                # lattice coordinates block (x increasing)
                for x in range(nx):
                    row.append(f"{x},{y},{z}")
                row.append("") 

                # cargo material
                for x in range(nx):
                    row.append(0)
                row.append("") 

                # cargo coordinates block (default 0,0,0)
                for x in range(nx):
                    row.append("0,0,0")
                rows.append(row)

        lattice_df = pd.DataFrame(rows)

        # --- 3. write to excel ---
        with pd.ExcelWriter(filename, engine="xlsxwriter") as writer:
            # config sheet: write title in A1, then table below
            config_df.to_excel(writer, sheet_name="Config", startrow=1, index=False)
            cfg_ws = writer.sheets["Config"]
            cfg_ws.write(0, 0, "Lattice Config")

        # Lattice sheet: no header/index
        lattice_df.to_excel(writer, sheet_name="Lattice", index=False, header=False)

        print(f"Saved lattice template to {filename}")

    @staticmethod
    def load_excel(filename="lattice.xlsx"):
        """Read lattice Excel file with sheets 'Config' and 'Lattice',
        returns a MOSES Lattice instance. Assumes the layout produced by create_excel()."""

        # --- 1. read config sheet ---
        config_df = pd.read_excel(filename, sheet_name="Config", header=1)
        nx = int(config_df.loc[config_df['param'] == 'nx', 'value'].values[0])
        ny = int(config_df.loc[config_df['param'] == 'ny', 'value'].values[0])
        nz = int(config_df.loc[config_df['param'] == 'nz', 'value'].values[0])
        is_unit_cell = bool(config_df.loc[config_df['param'] == 'is_unit_cell', 'value'].values[0])

        # --- 2. read lattice sheet ---
        lattice_df = pd.read_excel(filename, sheet_name="Lattice", header=None)
        voxels = []
        row_idx = 1
        for z in reversed(range(nz)):
            row_idx += 1  # skip layer label row
            for y in reversed(range(ny)):
                row = lattice_df.iloc[row_idx]
                # parse lattice coordinates
                for x in range(nx):
                    coord_str = row[x]
                    x_l, y_l, z_l = map(int, coord_str.split(","))
                    voxel_coords = (x_l, y_l, z_l)

                    # parse cargo material
                    cargo = int(row[nx + 1 + x])

                    # parse cargo coordinates
                    cargo_coord_str = row[2 * (nx + 1) + x]
                    cargo_x, cargo_y, cargo_z = map(int, cargo_coord_str.split(","))
                    cargo_coords = (cargo_x, cargo_y, cargo_z)

                    v = Voxel(
                        coords=voxel_coords,
                        cargo=cargo,
                        cargo_coords=cargo_coords
                    )
                    voxels.append(v)
                
                row_idx += 1

        lattice = Lattice(voxels=voxels, is_unit_cell=is_unit_cell)

        print(f"Loaded lattice from {filename}")
        return lattice
    
    @staticmethod
    def write_output_sheet(filename: str, voxels: list[Voxel], sheet_name: str = "Output"):
        """
        Append an 'Output' sheet to an existing lattice Excel file.

        Parameters
        ----------
        filename : str
            Path to the existing Excel file (with Config and Lattice sheets).
        voxels : list[Voxel]
            List of Voxel objects that form the minimum origami (e.g., moses.mesovoxel.all_voxels).
        sheet_name : str, optional
            Name of the sheet to write (default 'Output').
        """
        # Map labels -> vertex directions used in Voxel.vertices
        dir_map = {
            "x+": (0.5, 0.0, 0.0),
            "x-": (-0.5, 0.0, 0.0),
            "y+": (0.0, 0.5, 0.0),
            "y-": (0.0, -0.5, 0.0),
            "z+": (0.0, 0.0, 0.5),
            "z-": (0.0, 0.0, -0.5),
        }

        rows = []
        for i, v in enumerate(voxels):
            x, y, z = v.coords
            row = {
                "Origami": v.id2,
                "Coordinates": f"{int(x)},{int(y)},{int(z)}",
                "Concentration": None, #TODO: later
                "Material": v.cargo
            }

            # fill directional bond colors; default to 0 if no bond or no color
            for label, vertex in dir_map.items():
                bond = v.get_bond(vertex)
                row[label] = bond.color if bond.color else 0

            rows.append(row)

        output_df = pd.DataFrame(
            rows,
            columns=[
                "Origami",
                "Coordinates",
                "Concentration",
                "Material",
                "x+",
                "x-",
                "y+",
                "y-",
                "z+",
                "z-",
            ],
        )

        # Write into the SAME Excel file, adding/replacing the Output sheet
        # Needs openpyxl: pip install openpyxl
        with pd.ExcelWriter(
            filename,
            engine="openpyxl",
            mode="a",
            if_sheet_exists="replace",
        ) as writer:
            # startrow=1 to put the title in row 1
            output_df.to_excel(writer, sheet_name=sheet_name, index=False, startrow=1)
            ws = writer.sheets[sheet_name]
            ws["A1"] = "Minimum Origami"


In [20]:
lat = LatticeLoader.load_excel("test.xlsx")
# now we run the moses algorithm
moses = Moses(lat)
moses.run() # 🏃‍♂️

Loaded lattice from test.xlsx


In [23]:
%gui qt

import sys
from PyQt6.QtWidgets import QApplication
from PyQt6.QtCore import QCoreApplication
from ui.Visualizer import RunVisualizer

if __name__ == '__main__':
    if not QCoreApplication.instance():
        app = QApplication(sys.argv)
    else:
        app = QCoreApplication.instance()

    visualizeWindow = RunVisualizer(lat.voxels, False, app)

In [25]:
# export to file
min_voxels = [lat.get_voxel(v) for v in moses.mesovoxel.all_voxels()]
LatticeLoader.write_output_sheet("test.xlsx", min_voxels, sheet_name="Output")